In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
df_2 = pd.read_csv('szereg_czasowy.csv', sep=";")

df_2 = df_2.dropna(subset=['retail_price'])
df_2 = df_2.reset_index(drop=True)
df_2 = df_2.sort_values(by='year')
df_2 = df_2.replace(',', '.')

df_2

In [ ]:
average_values = df_2.iloc[:, 8:].apply(lambda x: x.mean(), axis=0)
average_values_list = average_values.tolist()

average_values_rounded = [round(x, 6) for x in average_values_list]

average_values_rounded_series = pd.Series(average_values_rounded, name='average_values_rounded')
average_values_rounded_series.name = 'average_values_rounded'
df_2['average_values_rounded'] = average_values_rounded_series

LEGO_basket = pd.DataFrame(average_values_rounded_series)